In [5]:
# Tensorflow uses CPU instead of GPU device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [6]:
import pandas as pd

from SMILESX import main, inference
%matplotlib inline

RDKit WARNING: [12:54:19] Enabling RDKit 2019.09.3 jupyter extensions


#### Definition of data

In [7]:
validation_data_dir = "validation_data/"

In [8]:
extension = '.csv'

In [9]:
data_name = 'FreeSolv' # FreeSolv, ESOL, Lipophilicity
prop_tag = ''

In [10]:
if data_name == 'FreeSolv':
    data_filename = 'FreeSolv_SAMPL'
    prop_tag = 'expt'
elif data_name == 'ESOL':
    data_filename = 'ESOL_delaney-processed'
    prop_tag = 'measured log solubility in mols per litre'
elif data_name == 'Lipophilicity':
    data_filename = 'Lipophilicity'
    prop_tag = 'exp'
else:
    data_filename = data_name
    prop_tag = prop_tag

In [11]:
import os
try:
    data_file = os.path.join(validation_data_dir,data_filename+extension)
    sol_data = pd.read_csv(os.path.join(data_file))
except FileNotFoundError:
    print(f"file {data_file} doesn't exist")

In [12]:
sol_data.head(3)

,Unnamed: 0,iupac,smiles,expt,calc
0,0,"4-methoxy-N,N-dimethyl-benzamide",COc1ccc(C(=O)N(C)C)cc1,-11.01,-9.625
1,1,methanesulfonyl chloride,CS(=O)(=O)Cl,-4.87,-6.219
2,2,3-methylbut-1-ene,C=CC(C)C,1.83,2.452


In [13]:
sol_data = sol_data[['smiles',prop_tag]]

In [14]:
sol_data.head()

,smiles,expt
0,COc1ccc(C(=O)N(C)C)cc1,-11.01
1,CS(=O)(=O)Cl,-4.87
2,C=CC(C)C,1.83
3,CCc1cnccn1,-5.45
4,CCCCCCCO,-4.21


In [15]:
sol_data.shape

(642, 2)

#### Hyperparameters optimization with GPyOpt (Bayesian optimization)

In [16]:
dhyp_range = [int(2**itn) for itn in range(3,11)] # 
dalpha_range = [float(ialpha/10.) for ialpha in range(20,40,1)] # Adam's learning rate = 10^(-dalpha_range)

if data_name != 'Lipophilicity':
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'lrate', 'type': 'discrete', 'domain': dalpha_range}
    ]
else:
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': (1024, 1024)}, # fixed
        {'name': 'lrate', 'type': 'discrete', 'domain': (3, 3)} # fixed
    ]

In [ ]:
main.Main(data=sol_data, 
          data_name=data_name, 
          data_units='', 
          bayopt_bounds=bounds, 
          k_fold_number = 3, 
          augmentation = True, 
          outdir = "./data/", 
          bayopt_n_epochs = 1,
          bayopt_n_rounds = 1, 
          bayopt_it_factor = 1, 
          bayopt_on = True, 
          lstmunits_ref = 128, 
          denseunits_ref = 16, 
          embedding_ref = 32, 
          n_gpus = 1, 
          bridge_type = 'NVLink', 
          batch_size_ref = 8,
          alpha_ref = 22, 
          patience = 50, 
          n_epochs = 100)

***SMILES_X starts...***


******
***Fold #0 initiated...***
******
***Sampling and splitting of the dataset.***

Scaler: RobustScaler(copy=True, quantile_range=(5.0, 95.0), with_centering=True,
             with_scaling=True)
Train/valid/test splits: 0.80/0.10/0.10


***Data augmentation to True***

Enumerated SMILES:
	Training set: 4352
	Validation set: 474
	Test set: 529

***Tokenization of SMILES.***

Examples of tokenized SMILES from a training set:
[[' ', 'C', '=', 'C', 'C', 'C', ' '], [' ', 'C', '(', 'C', 'C', ')', '=', 'C', ' '], [' ', 'C', '(', 'C', ')', 'C', '=', 'C', ' '], [' ', 'C', 'C', 'C', '=', 'C', ' '], [' ', 'c', '1', 'c', 'c', 'c', '2', 'n', 'c', 'c', 'c', 'c', '2', 'c', '1', ' ']]

Number of tokens only present in a training set: 33

Number of tokens only present in a validation set: 22
Is the validation set a subset of the training set: True
What are the tokens by which they differ: set()

Number of tokens only present in a test set: 26
Is the test set a subset of 

Epoch 21/100
17/17 [==============================] - 50s 3s/step - loss: 0.0219 - mean_absolute_error: 0.1141 - mean_squared_error: 0.0219 - val_loss: 0.0251 - val_mean_absolute_error: 0.1309 - val_mean_squared_error: 0.0282
Epoch 22/100
17/17 [==============================] - 50s 3s/step - loss: 0.0187 - mean_absolute_error: 0.1078 - mean_squared_error: 0.0187 - val_loss: 0.0206 - val_mean_absolute_error: 0.1182 - val_mean_squared_error: 0.0204
Epoch 23/100
17/17 [==============================] - 50s 3s/step - loss: 0.0170 - mean_absolute_error: 0.1032 - mean_squared_error: 0.0170 - val_loss: 0.0169 - val_mean_absolute_error: 0.1104 - val_mean_squared_error: 0.0204
Epoch 24/100
17/17 [==============================] - 101s 6s/step - loss: 0.0185 - mean_absolute_error: 0.1073 - mean_squared_error: 0.0185 - val_loss: 0.0128 - val_mean_absolute_error: 0.1047 - val_mean_squared_error: 0.0172
Epoch 25/100
17/17 [==============================] - 483s 28s/step - loss: 0.0159 - mean_absol

17/17 [==============================] - 51s 3s/step - loss: 0.0068 - mean_absolute_error: 0.0629 - mean_squared_error: 0.0068 - val_loss: 0.0074 - val_mean_absolute_error: 0.0716 - val_mean_squared_error: 0.0088
Epoch 58/100
17/17 [==============================] - 51s 3s/step - loss: 0.0070 - mean_absolute_error: 0.0638 - mean_squared_error: 0.0070 - val_loss: 0.0067 - val_mean_absolute_error: 0.0658 - val_mean_squared_error: 0.0078
Epoch 59/100
17/17 [==============================] - 51s 3s/step - loss: 0.0064 - mean_absolute_error: 0.0608 - mean_squared_error: 0.0064 - val_loss: 0.0066 - val_mean_absolute_error: 0.0633 - val_mean_squared_error: 0.0071
Epoch 60/100
17/17 [==============================] - 51s 3s/step - loss: 0.0070 - mean_absolute_error: 0.0639 - mean_squared_error: 0.0070 - val_loss: 0.0082 - val_mean_absolute_error: 0.0726 - val_mean_squared_error: 0.0091
Epoch 61/100
17/17 [==============================] - 51s 3s/step - loss: 0.0066 - mean_absolute_error: 0.061

17/17 [==============================] - 52s 3s/step - loss: 0.0033 - mean_absolute_error: 0.0422 - mean_squared_error: 0.0033 - val_loss: 0.0055 - val_mean_absolute_error: 0.0531 - val_mean_squared_error: 0.0058
Epoch 94/100
17/17 [==============================] - 54s 3s/step - loss: 0.0037 - mean_absolute_error: 0.0457 - mean_squared_error: 0.0037 - val_loss: 0.0056 - val_mean_absolute_error: 0.0544 - val_mean_squared_error: 0.0066
Epoch 95/100
17/17 [==============================] - 52s 3s/step - loss: 0.0035 - mean_absolute_error: 0.0436 - mean_squared_error: 0.0035 - val_loss: 0.0050 - val_mean_absolute_error: 0.0577 - val_mean_squared_error: 0.0061
Epoch 96/100
17/17 [==============================] - 51s 3s/step - loss: 0.0030 - mean_absolute_error: 0.0410 - mean_squared_error: 0.0030 - val_loss: 0.0050 - val_mean_absolute_error: 0.0549 - val_mean_squared_error: 0.0065
Epoch 97/100
17/17 [==============================] - 51s 3s/step - loss: 0.0031 - mean_absolute_error: 0.041

268/268 [==============================] - 7s 25ms/step - loss: 0.0178 - mean_absolute_error: 0.1041 - mean_squared_error: 0.0178 - val_loss: 0.0690 - val_mean_absolute_error: 0.1426 - val_mean_squared_error: 0.0409
Epoch 15/100
268/268 [==============================] - 7s 25ms/step - loss: 0.0163 - mean_absolute_error: 0.1000 - mean_squared_error: 0.0163 - val_loss: 0.0643 - val_mean_absolute_error: 0.1217 - val_mean_squared_error: 0.0305
Epoch 16/100
268/268 [==============================] - 7s 25ms/step - loss: 0.0163 - mean_absolute_error: 0.0996 - mean_squared_error: 0.0163 - val_loss: 0.0629 - val_mean_absolute_error: 0.1289 - val_mean_squared_error: 0.0358
Epoch 17/100
268/268 [==============================] - 7s 25ms/step - loss: 0.0151 - mean_absolute_error: 0.0952 - mean_squared_error: 0.0151 - val_loss: 0.0623 - val_mean_absolute_error: 0.1379 - val_mean_squared_error: 0.0381
Epoch 18/100
268/268 [==============================] - 7s 25ms/step - loss: 0.0141 - mean_absolu

In [ ]:
pred_from_ens = inference.Inference(data_name=data_name, 
                                    smiles_list = ['CC','CCC','C=O','ABC','DEF'], 
                                    data_units = '',
                                    k_fold_number = 3,
                                    augmentation = True, 
                                    outdir = "./data/")

In [ ]:
pred_from_ens